In [1]:
!pip3 install transformers datasets sentencepiece peft evaluate -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 87.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.12.0 which is incompatible.
bigframes 

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

import torch
import torch.optim as optim
import torch.nn as nn
import re
from transformers import (
    GPTNeoXConfig, T5Config, LlamaConfig, AutoTokenizer, AutoModelForCausalLM, DataCollatorWithPadding, AutoConfig, AutoModelForSequenceClassification
)
from transformers import logging as hf_logging
import torch.nn.functional as F

from datasets import Dataset, load_dataset, concatenate_datasets
from dataclasses import dataclass
from tqdm import tqdm

import transformers
from datasets import Dataset
from sklearn.metrics import roc_auc_score

import evaluate
import time
from torch.nn.functional import sigmoid

metric = evaluate.load('accuracy')



2025-04-28 05:41:51.954200: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745818912.199544      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745818912.264018      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
df_ai  = pd.read_csv('/kaggle/input/ai-human/ai_generated_train_essays.csv')
ai_500_samples = pd.read_csv('/kaggle/input/ai-human/ai_generated_train_essays.csv')
persuade_df = pd.read_csv('/kaggle/input/ai-human/persuade_2.0_human_scores_demo_id_github.csv')
df_llama = pd.read_csv('/kaggle/input/ai-human/llama_70b_v1.csv')
df_falcon = pd.read_csv('/kaggle/input/ai-human/falcon_180b_v1.csv')
proper_train = pd.read_csv('/kaggle/input/ai-human/train_drcat_01.csv')
aug_df = pd.read_csv('/kaggle/input/ai-human/argugpt.csv')
corrected_df = pd.read_csv('/kaggle/input/ai-human/corrected_persuade_10k.csv')


aug_df = pd.DataFrame ({'text' : aug_df['text'], 'generated':1})
propertrain_ai = pd.DataFrame({'text' : proper_train['text'] [proper_train ['label'] ==1], 'generated' : 1}) .reset_index (drop=True)
df_nbroad= pd.concat ([df_llama, df_falcon]) .reset_index(drop=True)
nbroad_ai= pd.DataFrame ({'text' : df_nbroad['generated_text'], 'generated' : 1}) .reset_index(drop=True)
ai_ds= pd.concat([propertrain_ai, nbroad_ai, ai_500_samples.drop(columns=['prompt_id' , 'id']), aug_df]).reset_index(drop=True)
hooman_ds = pd.DataFrame({'text' : corrected_df ['corrected_text'], 'generated' : 0}) .reset_index(drop=True)
# final_df = pd.concat([ai_ds.sample(n=500, random_state=42) , hooman_ds.sample(n=500, random_state=42)]).sample (frac=1).reset_index(drop=True)
final_df = pd.concat([ai_ds , hooman_ds]).sample (frac=1).reset_index(drop=True)
final_df

,text,generated
0,Summer is anticipated by eager students across...,0
1,Character Traits: Are They Chosen or Formed?\n...,1
2,I think that meditation and mindfulness practi...,1
3,"As an eighth-grade student, I believe that rel...",1
4,"Ummm... Okay! So, like, failure and mistakes....",1
...,...,...
24023,There has always been extracurricular activiti...,0
24024,"As an eighth-grade student, I believe that enf...",1
24025,Advantages and Disadvantages of Working Alone ...,1
24026,Freedom without a car\n\nAround the world in d...,0


In [4]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained('distilbert/distilbert-base-uncased')
from transformers import Trainer, TrainingArguments
model_name = 'distilbert/distilbert-base-uncased'

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

def preprocess_function(examples):
    return tokenizer(examples['text'], max_length=512, padding='max_length', truncation=True)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [6]:
ds = Dataset.from_pandas(final_df).rename_column("generated", "label").train_test_split(test_size=0.15)
ds['train'] = ds['train'].map(preprocess_function, batched=False, num_proc=16, remove_columns=['text'])
ds['test'] = ds['test'].map(preprocess_function, batched=False, num_proc=16, remove_columns=['text'])
ds

Map (num_proc=16):   0%|          | 0/20423 [00:00<?, ? examples/s]

Map (num_proc=16):   0%|          | 0/3605 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 20423
    })
    test: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 3605
    })
})

In [7]:
FLAGS = {'MAX_INPUT': 512,
         'LOGGING_STEPS': 100,
         'NUM_EPOCHS': 10,
         'BATCH_SIZE': 16, #Making batch_size lower then 8 will result in slower training, but will allow for larger models\context. Fortunately, we have 128GBs. Setting higher batch_size doesn't seem to improve time.
          'NUM_STEPS': len(ds)}

print(torch.cuda.is_available())  # Should return True if GPU is enabled


True


In [8]:
training_loader = torch.utils.data.DataLoader(ds['train'], batch_size=FLAGS['BATCH_SIZE'], collate_fn=DataCollatorWithPadding(tokenizer=tokenizer))
testing_loader = torch.utils.data.DataLoader(ds['test'], batch_size=FLAGS['BATCH_SIZE'], collate_fn=DataCollatorWithPadding(tokenizer=tokenizer))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [9]:
import evaluate
def train(FLAGS):
    accuracy_metric = evaluate.load("accuracy") 
    num_iterations = int(FLAGS['NUM_STEPS'] / FLAGS['BATCH_SIZE'])
    lr = 1e-5
    optimizer = optim.Adam(model.parameters(), lr=lr)
    scheduler = torch.optim.lr_scheduler.LinearLR(optimizer, start_factor=1.0, end_factor=0.1, total_iters=FLAGS['NUM_STEPS'] * FLAGS['BATCH_SIZE'])
    model.to(device)
    accuracy_metric = evaluate.load("accuracy")
    start_time = time.time()

    for epoch in range(1, FLAGS['NUM_EPOCHS'] + 1):
        model.train()  # Important! Set training mode at the start of epoch

        for step, batch in enumerate(training_loader):
            optimizer.zero_grad()

            input_ids, attention_mask, labels = batch.input_ids.to(device), batch.attention_mask.to(device), torch.unsqueeze(batch.labels.to(device), 1)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            loss.backward()
            optimizer.step()
            scheduler.step()

            if (step + 1) % FLAGS['LOGGING_STEPS'] == 0:
                elapsed_time = time.time() - start_time
                print(f'Epoch [{epoch}], Step [{step+1}], Loss: {loss.item():.4f}, Time elapsed: {elapsed_time:.2f}s')

                # NOW Validation phase
                model.eval()
                total_loss = 0.0
                total_steps = 0
                all_predictions = []
                all_labels = []

                with torch.no_grad():
                    for batch in testing_loader:
                        input_ids, attention_mask, labels = batch.input_ids.to(device), batch.attention_mask.to(device), torch.unsqueeze(batch.labels.to(device), 1)

                        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
                        loss = outputs.loss

                        total_loss += loss.item()
                        total_steps += 1

                        predictions = torch.argmax(outputs.logits, dim=1)
                        all_predictions.extend(predictions.cpu().numpy())
                        all_labels.extend(labels.cpu().numpy())

                average_loss = total_loss / total_steps
                accuracy_result = accuracy_metric.compute(predictions=all_predictions, references=all_labels)

                print(f"Validation Loss after {step+1} steps: {average_loss:.4f}")
                print(f"Validation Accuracy after {step+1} steps: {accuracy_result['accuracy']:.4f}")

                model.train()  # VERY IMPORTANT: Switch back to training mode after validation


In [10]:
train(FLAGS)

Epoch [1], Step [100], Loss: 0.3564, Time elapsed: 49.83s
Validation Loss after 100 steps: 0.3972
Validation Accuracy after 100 steps: 0.9345
Epoch [1], Step [200], Loss: 0.2007, Time elapsed: 129.69s
Validation Loss after 200 steps: 0.2582
Validation Accuracy after 200 steps: 0.9537
Epoch [1], Step [300], Loss: 0.1312, Time elapsed: 210.24s
Validation Loss after 300 steps: 0.1753
Validation Accuracy after 300 steps: 0.9650
Epoch [1], Step [400], Loss: 0.1134, Time elapsed: 290.77s
Validation Loss after 400 steps: 0.1452
Validation Accuracy after 400 steps: 0.9626
Epoch [1], Step [500], Loss: 0.0626, Time elapsed: 371.22s
Validation Loss after 500 steps: 0.0865
Validation Accuracy after 500 steps: 0.9853
Epoch [1], Step [600], Loss: 0.0557, Time elapsed: 451.71s
Validation Loss after 600 steps: 0.0845
Validation Accuracy after 600 steps: 0.9809
Epoch [1], Step [700], Loss: 0.0404, Time elapsed: 532.26s
Validation Loss after 700 steps: 0.0882
Validation Accuracy after 700 steps: 0.9742


In [11]:
model.eval()

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


In [12]:
def evaluate_final_accuracy():
    model.eval()  # Set the model to evaluation mode (no training updates)

    total_loss = 0.0
    total_steps = 0
    all_predictions = []
    all_labels = []

    with torch.no_grad():  # No need to calculate gradients for evaluation
        for batch in testing_loader:
            input_ids, attention_mask, labels = batch.input_ids.to(device), batch.attention_mask.to(device), torch.unsqueeze(batch.labels.to(device), 1)

            # Forward pass
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss

            total_loss += loss.item()
            total_steps += 1

            # Collect predictions and labels
            predictions = torch.argmax(outputs.logits, dim=1)
            all_predictions.extend(predictions.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Calculate average loss
    final_loss = total_loss / total_steps

    # Calculate final accuracy
    accuracy_metric = evaluate.load("accuracy")
    final_accuracy = accuracy_metric.compute(predictions=all_predictions, references=all_labels)

    print(f"Final Validation Loss: {final_loss:.4f}")
    print(f"Final Validation Accuracy: {final_accuracy['accuracy']:.4f}")

# Call this function to get final accuracy
evaluate_final_accuracy()

Final Validation Loss: 0.0498
Final Validation Accuracy: 0.9883
